In [229]:
import pandas as pd
from textblob import TextBlob
from deep_translator import GoogleTranslator
import nltk, re, random
df_tweets = pd.read_json("versailles_tweets_100.json")
df_tweets

,_id,public_metrics,id,conversation_id,author_id,text,geo,lang,created_at,entities,context_annotations,attachments
0,1421616335700824064,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421616335700824064,1421616335700824064,1339914264522461184,Goumin des éléphants joueurs la même fatigue m...,{'place_id': '00b8943291443c8c'},fr,2021-07-31 23:38:41+00:00,"{'hashtags': [{'start': 52, 'end': 63, 'tag': ...",NaN,NaN
1,1421599703116943360,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421599703116943360,1421599703116943360,1339914264522461184,@ericbailly24 @maxigr04del mes tontons vous a...,{'place_id': '00b8943291443c8c'},fr,2021-07-31 22:32:35+00:00,"{'mentions': [{'start': 0, 'end': 13, 'usernam...","[{'domain': {'id': '6', 'name': 'Sports Event'...",NaN
2,1421599163561742336,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",1421599163561742336,1421599163561742336,1339914264522461184,Ah oui le sommeil là sera compliqué. #CIV est...,{'place_id': '00b8943291443c8c'},fr,2021-07-31 22:30:27+00:00,"{'hashtags': [{'start': 37, 'end': 41, 'tag': ...","[{'domain': {'id': '6', 'name': 'Sports Event'...",NaN
3,1421591889095057408,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421591889095057408,1421591889095057408,1339914264522461184,"31 juillet , journée internationale de la femm...",{'place_id': '00b8943291443c8c'},fr,2021-07-31 22:01:32+00:00,"{'hashtags': [{'start': 60, 'end': 65, 'tag': ...",NaN,NaN
4,1421582795294617600,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421582795294617600,1421582795294617600,717025418,Le pedigree 😂😂😂😂 https://t.co/D3Rv7A2BOF,{'place_id': '00b8943291443c8c'},es,2021-07-31 21:25:24+00:00,"{'urls': [{'start': 17, 'end': 40, 'url': 'htt...",NaN,NaN
5,1421581383454052352,"{'retweet_count': 2, 'reply_count': 0, 'like_c...",1421581383454052352,1421256177359466496,992904738516717568,@isabelle170516 @leonna_julie @Steiner2502 Vou...,{'place_id': '011a304d683c2461'},fr,2021-07-31 21:19:48+00:00,"{'mentions': [{'start': 0, 'end': 15, 'usernam...",NaN,NaN
6,1421575939700445184,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421575939700445184,1421256177359466496,992904738516717568,@LynLyna12 @leonna_julie La grande muette cont...,{'place_id': '011a304d683c2461'},fr,2021-07-31 20:58:10+00:00,"{'mentions': [{'start': 0, 'end': 10, 'usernam...",NaN,NaN
7,1421569996858269696,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421569996858269696,1421569996858269696,736523371,Under wsh 😳😳😳,{'place_id': '011a304d683c2461'},en,2021-07-31 20:34:33+00:00,NaN,NaN,NaN
8,1421563798503272448,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421563798503272448,1421563798503272448,1471684208,Les bains d'apollon #versailles #nocturne #app...,"{'place_id': '011a304d683c2461', 'coordinates'...",fr,2021-07-31 20:09:55+00:00,"{'urls': [{'start': 76, 'end': 99, 'url': 'htt...",NaN,NaN
9,1421562928176803840,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",1421562928176803840,1421553997811421184,992904738516717568,@leonna_julie Le rdv aujourd'hui aura tenu se...,{'place_id': '011a304d683c2461'},fr,2021-07-31 20:06:28+00:00,"{'mentions': [{'start': 0, 'end': 13, 'usernam...",NaN,{'media_keys': ['3_1421562925999869960']}


In [230]:
test = df_tweets.iloc[1]
test

_id                                                  1421599703116943360
public_metrics         {'retweet_count': 0, 'reply_count': 0, 'like_c...
id                                                   1421599703116943360
conversation_id                                      1421599703116943360
author_id                                            1339914264522461184
text                   @ericbailly24 @maxigr04del  mes tontons vous a...
geo                                     {'place_id': '00b8943291443c8c'}
lang                                                                  fr
created_at                                     2021-07-31 22:32:35+00:00
entities               {'mentions': [{'start': 0, 'end': 13, 'usernam...
context_annotations    [{'domain': {'id': '6', 'name': 'Sports Event'...
attachments                                                          NaN
Name: 1, dtype: object

In [231]:
def text_cleaning(text:str):
    text = re.sub(r'^\ +|@\w+|#\w+','',text)
    text = ''.join(e for e in text if e.isalnum() or e.isspace() or e == "," or e == ".")
    return re.sub(r'\  +',' ',text)

def author(publi):
    return publi["author_id"]

def hashtags(publi):
    output = []
    for i in publi["entities"]["hashtags"]:
         output.append(i["tag"])
    return output

def mentions(publi):
    output = []
    for i in publi["entities"]["mentions"]:
         output.append(i["username"])
    return output if len(output) > 0 else None

def sentiment_(publi):
    return TextBlob(publi["text"]).sentiment

def topics(publi):
    return list(TextBlob(GoogleTranslator(source='fr', target='en').translate(text_cleaning(publi['text']))).noun_phrases)


author(test)
hashtags(test)
mentions(test)
sentiment_(test)
topics(test)

print(test['text'])

@ericbailly24 @maxigr04del  mes tontons vous avez fait votre part , JO prochain on ira en demi final au moins. BRAVO à vous . #SupportriceMazo #domie #CIV


In [248]:
def top_K_hashtags(k):
    hashtags_list = []
    for i in df_tweets['entities']:
        hashtags_list += i['hashtags']['tag']
    return hashtags_list

#top_K_hashtags(1)

for i in df_tweets['entities']:
    print(i.keys())

dict_keys(['hashtags'])
dict_keys(['mentions', 'hashtags'])
dict_keys(['hashtags'])
dict_keys(['hashtags'])
dict_keys(['urls'])
dict_keys(['mentions'])
dict_keys(['mentions'])


AttributeError: 'float' object has no attribute 'keys'